In [1]:
import os
import getpass


from utils import *
from keras.models import Sequential, Model, load_model
from keras.layers import Flatten, Dense, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pickle as cPickle

In [2]:
from matplotlib import pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score,confusion_matrix,multilabel_confusion_matrix,recall_score
from sklearn.preprocessing import normalize
from sklearn import svm
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [ ]:
my_first_mlp_imgsize64_batchsize64_4layers_2048_512_128.h5

In [3]:
#user defined variables
version='imgsize64_batchsize64_4layers_2048_512_128'
IMG_SIZE   = 64 #deafult 32
BATCH_SIZE  = 64 #default 16
DATASET_DIR = '../../../Databases/MIT_split'#'/home/mcv/datasets/MIT_split'
MODEL_FNAME = f'output_results/my_first_mlp_imgsize64_batchsize64_4layers_2048_512_128.h5'

# if not os.path.exists(DATASET_DIR):
#   print(Color.RED, 'ERROR: dataset directory '+DATASET_DIR+' does not exist!\n')
#   quit()


print('Building MLP model...\n')

#Build the Multi Layer Perceptron model

####
# Split into tiles
####

model = Sequential()
model.add(Reshape((IMG_SIZE*IMG_SIZE*3,),input_shape=(IMG_SIZE, IMG_SIZE, 3),name='first'))
model.add(Dense(units=2048, activation='relu',name='second'))
model.add(Dense(units=512, activation='relu', name='3'))
model.add(Dense(units=128, activation='relu', name='4'))
model.add(Dense(units=8, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

print(model.summary())
#plot_model(model, to_file=f'output_results/feature_extraction_{version}.png', show_shapes=True, show_layer_names=True)

model.load_weights(MODEL_FNAME)
#plot_model(model, to_file=f'output_results/feature_extraction_{version}.png', show_shapes=True, show_layer_names=True)

print('Done!\n')

Building MLP model...

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first (Reshape)              (None, 12288)             0         
_________________________________________________________________
second (Dense)               (None, 2048)              25167872  
_________________________________________________________________
3 (Dense)                    (None, 512)               1049088   
_________________________________________________________________
4 (Dense)                    (None, 128)               65664     
_________________________________________________________________
dense (Dense)                (None, 8)                 1032      
Total params: 26,283,656
Trainable params: 26,283,656
Non-trainable params: 0
_________________________________________________________________
None
Done!



In [7]:
model_layer = Model(inputs=model.input, outputs=model.get_layer('4').output)

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [4]:
train_images_filenames = cPickle.load(open('resources/train_images_filenames.dat','rb'))
test_images_filenames = cPickle.load(open('resources/test_images_filenames.dat','rb'))
train_labels = cPickle.load(open('resources/train_labels.dat','rb'))
test_labels = cPickle.load(open('resources/test_labels.dat','rb'))

In [5]:
train_images_filenames[0]
len(Train_descriptors[0][0].shape)

NameError: name 'Train_descriptors' is not defined

In [8]:
#get the features from images

Train_descriptors = []
Train_label_per_descriptor = []


for filename,labels in zip(train_images_filenames,train_labels):
    x = np.asarray(Image.open(filename))
    x = np.expand_dims(np.resize(x, (IMG_SIZE, IMG_SIZE, 3)), axis=0)
#     print('prediction for image ' + filename)
    Train_descriptors.append(model_layer.predict(x/255.0))
    Train_label_per_descriptor.append(labels)
#     print(features)
#     print('Done!')

In [9]:
Test_descriptors = []
Test_label_per_descriptor = []


for filename,labels in zip(test_images_filenames,test_labels):
    x = np.asarray(Image.open(filename))
    x = np.expand_dims(np.resize(x, (IMG_SIZE, IMG_SIZE, 3)), axis=0)
#     print('prediction for image ' + filename)
    Test_descriptors.append(model_layer.predict(x/255.0))
    Test_label_per_descriptor.append(labels)
#     print(features)
#     print('Done!')

### Linear kernel only SVM without codebook

In [10]:
Train_descriptors=[i[0] for i in Train_descriptors]

In [11]:
Test_descriptors=[i[0] for i in Test_descriptors]

In [ ]:
lin_clf = svm.LinearSVC(max_iter=2000, C=10)
lin_clf.fit(Train_descriptors, train_labels)

scores = cross_val_score(lin_clf, Test_descriptors, Test_label_per_descriptor, cv=5)
accuracy=scores.mean()*100

print(accuracy)

### RBF Kernel only SVM without codebook

In [ ]:
rbf_svc = svm.SVC(kernel='rbf', C=1000, gamma=0.001)
rbf_svc.fit(Train_descriptors, train_labels)

scores = cross_val_score(rbf_svc, Test_descriptors, test_labels, cv=5)

accuracy=scores.mean()*100

print(accuracy)

### BoW with Linear Kernel

In [ ]:
D=np.vstack(Train_descriptors)

k=128
codebook = MiniBatchKMeans(n_clusters=k, verbose=False, batch_size=k * 20,compute_labels=False,reassignment_ratio=10**-4,random_state=42)
codebook.fit(D)

visual_words=np.zeros((len(Train_descriptors),k),dtype=np.float32)
for i in range(len(Train_descriptors)):
    words=codebook.predict(Train_descriptors[i])
    visual_words[i,:]=normalize(np.bincount(words,minlength=k).reshape(1,-1))

# knn = KNeighborsClassifier(n_neighbors=37,n_jobs=-1,metric='manhattan')
# knn.fit(visual_words, train_labels)

today = datetime.now()
dt_string = today.strftime("%H:%M:%S")
print(f"{dt_string} started doing")


In [ ]:
scaler = StandardScaler()
scaler.fit_transform(visual_words)

In [ ]:
lin_clf = svm.LinearSVC(max_iter=2000, C=10)
lin_clf.fit(visual_words, train_labels)

visual_words_test=np.zeros((len(test_images_filenames),k),dtype=np.float32)
for i in range(len(test_images_filenames)):
    filename=test_images_filenames[i]

    x = np.asarray(Image.open(filename))
    x = np.expand_dims(np.resize(x, (IMG_SIZE, IMG_SIZE, 3)), axis=0)
#     print('prediction for image ' + filename)
    des = model_layer.predict(x/255.0)
    words=codebook.predict(des)
    visual_words_test[i,:]=normalize(np.bincount(words,minlength=k).reshape(1,-1))

visual_words_test=scaler.transform(visual_words_test)
scores = cross_val_score(lin_clf, visual_words_test, test_labels, cv=5)
accuracy=scores.mean()*100

print(accuracy)

### BoW with RBF Kernel

In [ ]:
D=np.vstack(Train_descriptors)

k=128
codebook = MiniBatchKMeans(n_clusters=k, verbose=False, batch_size=k * 20,compute_labels=False,reassignment_ratio=10**-4,random_state=42)
codebook.fit(D)

visual_words=np.zeros((len(Train_descriptors),k),dtype=np.float32)
for i in range(len(Train_descriptors)):
    words=codebook.predict(Train_descriptors[i])
    visual_words[i,:]=normalize(np.bincount(words,minlength=k).reshape(1,-1))

# knn = KNeighborsClassifier(n_neighbors=37,n_jobs=-1,metric='manhattan')
# knn.fit(visual_words, train_labels)

today = datetime.now()
dt_string = today.strftime("%H:%M:%S")
print(f"{dt_string} started doing")


In [ ]:
scaler = StandardScaler()
scaler.fit_transform(visual_words)

In [ ]:
rbf_svc = svm.SVC(kernel='rbf', C=1000, gamma=0.001)
rbf_svc.fit(visual_words, train_labels)

visual_words_test=np.zeros((len(test_images_filenames),k),dtype=np.float32)
for i in range(len(test_images_filenames)):
    filename=test_images_filenames[i]

    x = np.asarray(Image.open(filename))
    x = np.expand_dims(np.resize(x, (IMG_SIZE, IMG_SIZE, 3)), axis=0)
#     print('prediction for image ' + filename)
    des = model_layer.predict(x/255.0)
    words=codebook.predict(des)
    visual_words_test[i,:]=normalize(np.bincount(words,minlength=k).reshape(1,-1))

visual_words_test=scaler.transform(visual_words_test)
scores = cross_val_score(rbf_svc, visual_words_test, test_labels, cv=5)

accuracy=scores.mean()*100

print(accuracy)